In [25]:
#!pip install qpsolvers

  Using cached Cython-0.29.22-cp37-cp37m-win_amd64.whl (1.6 MB)
  Created wheel for qpsolvers: filename=qpsolvers-1.5-py3-none-any.whl size=18784 sha256=555108b1af4667846f602750530b99fb42207697d2fa1397b3cce9d8c8f8a509
  Stored in directory: c:\users\louis\appdata\local\pip\cache\wheels\67\0f\3d\8568baeb3a81342ce737d1a387eafa48a909400c4efa63e232
  Created wheel for quadprog: filename=quadprog-0.1.8-cp37-cp37m-win_amd64.whl size=89564 sha256=b7612c58169a6b99d200d04604e17a39b18fdf499eedbbc9429a627ed53bd852
  Stored in directory: c:\users\louis\appdata\local\pip\cache\wheels\40\74\3a\072734cebfae52efc8a2523fd4703da7dfac29f17bd01ee372
Successfully built qpsolvers quadprog


In [43]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg
from tqdm import tqdm
import numba 
from numba import njit, vectorize, jit
import time 
import scipy

import qpsolvers
from qpsolvers import solve_qp

In [9]:
X_train = pd.read_csv('data/Xtr0_mat100.csv',delimiter= ' ', header= None)
Y_train = pd.read_csv('data/Ytr0.csv',delimiter= ',')
X_test = pd.read_csv('data/Xte0_mat100.csv',delimiter= ' ', header= None)

In [10]:
X = X_train.values
Y = Y_train[['Bound']].values

In [27]:
print(Y.shape)

(2000, 1)


In [15]:
@njit
def GaussianKernel(x, y, sig2 = 1): 
    return np.exp(-numpy.linalg.norm(x-y)**2/(2*sig2))

In [24]:
time1 = time.time()
Kernel = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : GaussianKernel(x2, x1), 1, X), 1, X)
time2 = time.time()
print(time2 - time1, 's')

16.318369150161743


In [19]:
print(Kernel)

[[1.         0.98475818 0.98371162 ... 0.98848832 0.98942306 0.98895558]
 [0.98475818 1.         0.9799995  ... 0.98545651 0.98173781 0.97988373]
 [0.98371162 0.9799995  1.         ... 0.97826428 0.98196982 0.97699373]
 ...
 [0.98848832 0.98545651 0.97826428 ... 1.         0.98708787 0.98545651]
 [0.98942306 0.98173781 0.98196982 ... 0.98708787 1.         0.98650493]
 [0.98895558 0.97988373 0.97699373 ... 0.98545651 0.98650493 1.        ]]


In [5]:
def hinge_loss(u):
    return max([0, 1-u])


    

In [7]:
class SVM:
    def __init__(self, kernel, C):
        self.kernel = kernel
        self.C = C
        
        
    def fit(self, X, Y):
        K = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : GaussianKernel(x2, x1), 1, X), 1, X)
        
        solver = solve_qp(K, Y, )
        

0


In [44]:
print(qpsolvers.available_solvers)

['quadprog']


In [58]:
Y_bis = np.ravel(Y) #diminue la dimension de (n,1) à (n,)

n = 2000
lbd = 1
C = 1 / (2 * n * lbd)

'''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
ça revient au même et je crois que le solver devrait fonctionner avec ça, mais j'y arrive pas encore
'''
G = np.vstack((np.diag(Y_bis), -np.diag(Y_bis)))

h = np.ones(2*n)
h[:n] = h[:n] * C
h[n:] = h[:n] * 0

#Si j'ai bien compris mettre un vecteur de 1 pour A et 1 pour b permet de pas les prendre en compte 
#(ici on a pas d'égalité)
A = np.ones(n)
b = np.array([1.])

tentative = solve_qp(Kernel, Y_bis, G, h, A, b, solver = 'quadprog')

print(tentative)

ValueError: Buffer dtype mismatch, expected 'double' but got 'long long'

### Ancien pour Ridge et test:

In [59]:
n = 2000
lbd = 1
C = 1 / (2 * n * lbd)

h = np.ones(n)*C

G = np.diag(Y_bis)

tentative = solve_qp(Kernel, Y_bis, G, h, A, b, solver = 'quadprog')

print(tentative)

ValueError: Buffer dtype mismatch, expected 'double' but got 'long long'

In [45]:
lbd = 3

#w_pas_benef = X.T.dot(np.linalg.inv(X.dot(X.T) + lbd * np.eyes(2000))).dot(Y)
def KRR(X,Y):
    n, d = x.shape
    w = np.linalg.inv(X.T.dot(X) + lbd * n * np.eye(d)).dot(X.T.dot(Y))
    return w


result = X@w_benef